# 3_GitlabRunner_ShellTest – Running GitLab CI Jobs Locally

This notebook demonstrates how to use `gitlab-runner` with the `shell` executor to test CI jobs directly on your local system. It enables fast, local validation of `.gitlab-ci.yml` configurations without requiring Docker or pushing to a remote GitLab server.

In [1]:
import subprocess
import os
from pathlib import Path

def in_jupyter():
    try:
        get_ipython
        return True
    except NameError:
        return False

# ------------------------------------------------------------------
# Paths and environment
# ------------------------------------------------------------------
repo_root = Path.cwd()
proj = repo_root / "gitlab_demo_shell"
proj.mkdir(parents=True, exist_ok=True)

env = os.environ.copy()
env["SHELL"] = "/bin/bash"

# macOS Homebrew path (adjust if needed)
env["PATH"] = "/opt/homebrew/bin:/usr/local/bin:" + env["PATH"]

# ------------------------------------------------------------------
# Test file
# ------------------------------------------------------------------
(proj / "test_sample.py").write_text(
"""\
def add(a, b):
    return a + b

def test_add():
    assert add(2, 2) == 4
"""
)

# ------------------------------------------------------------------
# GitLab CI configuration
# ------------------------------------------------------------------
(proj / ".gitlab-ci.yml").write_text(
"""\
stages:
  - test

pytest:
  stage: test
  script:
    - python -m pip install --quiet pytest
    - pytest
"""
)

# ------------------------------------------------------------------
# Git repository (idempotent)
# ------------------------------------------------------------------
if not (proj / ".git").exists():
    subprocess.run("git init", shell=True, cwd=proj, env=env)
    subprocess.run("git branch -m main", shell=True, cwd=proj, env=env)
    subprocess.run("git config user.name 'CI Tester'", shell=True, cwd=proj, env=env)
    subprocess.run("git config user.email 'ci@test.local'", shell=True, cwd=proj, env=env)
    subprocess.run("git add .", shell=True, cwd=proj, env=env)
    subprocess.run("git commit -m 'initial commit'", shell=True, cwd=proj, env=env)

# ------------------------------------------------------------------
# Jupyter hint
# ------------------------------------------------------------------
if in_jupyter():
    print("ℹ️ Running inside Jupyter.")
    print("   CI jobs will be executed locally using `gitlab-ci-local`.")
    print("   Missing git remotes are normal in this setup.\n")
    print("🔁 To run manually in a terminal:")
    print(f"   cd {proj}")
    print("   gitlab-ci-local\n")

# ------------------------------------------------------------------
# Run CI locally
# ------------------------------------------------------------------
print(f"\n🚀 Running GitLab CI pipeline locally in: {proj}\n")

try:
    subprocess.run(
        "gitlab-ci-local",
        shell=True,
        cwd=proj,
        env=env,
        check=True
    )
    print("\n✅ GitLab CI pipeline ran successfully.")
except subprocess.CalledProcessError:
    print("\n❌ GitLab CI pipeline failed.")
    print("   If this happens, try running the command manually in a terminal.")


Leeres Git-Repository in /Users/rpotthas/all/e-ai_ml2/course/code/code14/gitlab_demo_shell/.git/ initialisiert
[main (Root-Commit) 369ac7e] initial commit
 2 files changed, 13 insertions(+)
 create mode 100644 .gitlab-ci.yml
 create mode 100644 test_sample.py
ℹ️ Running inside Jupyter.
   CI jobs will be executed locally using `gitlab-ci-local`.
   Missing git remotes are normal in this setup.

🔁 To run manually in a terminal:
   cd /Users/rpotthas/all/e-ai_ml2/course/code/code14/gitlab_demo_shell
   gitlab-ci-local


🚀 Running GitLab CI pipeline locally in: /Users/rpotthas/all/e-ai_ml2/course/code/code14/gitlab_demo_shell



Unable to retrieve default remote branch, falling back to `main`.
  Command failed with exit code 128: git symbolic-ref --short refs/remotes/origin/HEAD
Schwerwiegend: ref refs/remotes/origin/HEAD is not a symbolic ref
Using fallback git remote data
  Command failed with exit code 2: git remote get-url origin
Fehler: Remote-Repository 'origin' nicht gefunden
parsing and downloads finished in 31 ms.
json schema validated in 71 ms


pytest starting shell (test)
pytest $ python -m pip install --quiet pytest
pytest $ pytest
pytest > ============================= test session starts ==============================
pytest > platform darwin -- Python 3.12.12, pytest-9.0.2, pluggy-1.6.0
pytest > rootdir: /Users/rpotthas/all/e-ai_ml2/course/code/code14/gitlab_demo_shell
pytest > plugins: anyio-4.12.0, langsmith-0.5.0
pytest > collected 1 item
pytest > 
pytest > test_sample.py .                                                         [100%]
pytest > 
pytest > ============================== 1 passed in 0.03s ===============================
pytest finished in 3.72 s

 PASS  pytest

✅ GitLab CI pipeline ran successfully.


pipeline finished in 3.87 s
